This notebook uses compiled version of the computeVPB function. To compile it run the command

    python setup.py build_ext --inplace

from the `python/` directory

See https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html for some basic tutorial

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ripser
import persim


In [2]:
import TDAvec
from TDAvec import pmin, pmax, DiagToPD, \
    computeVPB, computePL, computePS, computeNL, computeVAB

In [3]:
X = np.loadtxt("../R/unitCircle.csv", skiprows=1, delimiter=",")
D = ripser.ripser(X, thresh=2)["dgms"]
D[0][-1, 1] = 2


## computeVPB

In [4]:
PD = DiagToPD(D)
ySeqH0 = np.quantile(PD[0][1], np.arange(0, 1.1, 0.2))
xSeqH1 = np.quantile(PD[1][0], np.arange(0, 1.1, 0.2))
ySeqH1 = np.quantile(PD[1][1], np.arange(0, 1.1, 0.2))

vpb0 = computeVPB(PD, homDim=0, xSeq=[], ySeq=ySeqH0)
print(np.allclose(
    vpb0, 
    np.loadtxt("../R/vpb_0.csv", skiprows=1)
    ))
vpb1 = computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)
vpb1 = np.transpose(vpb1).reshape( (25,))
print(np.allclose(
    vpb1, 
    np.loadtxt("../R/vpb_1.csv", skiprows=1)
    , atol=1e-7))


True
True


## Others

In [44]:
scaleSeq = np.linspace(0, 2, 11)
# universal function for comparison
def compareResults(func, R_prefix, maxDim = 1, D=D, scaleSeq = scaleSeq, atol = 1e-7):
    print(f"Comparing compute{R_prefix.upper()}:")
    # for each of the dims
    for d in range(maxDim+1):
        # calc python result
        pyth = func(D, d, scaleSeq)
        # read R result
        R = np.loadtxt(f"../R/{R_prefix}_{d}.csv", skiprows=1)
        # report if they are equal
        print("\t dim=", d, ":", np.allclose(pyth, R, atol=atol))


In [53]:
testDict = {"pl":computePL, "ps":computePS, "nl":computeNL, "vab":computeVAB, "ecc":computeECC}
for p in testDict.keys():
    compareResults(testDict[p], p)

Comparing computePL:
	 dim= 0 : True
	 dim= 1 : True
Comparing computePS:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeNL:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeVAB:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeECC:
	 dim= 0 : True
	 dim= 1 : True
